In [ ]:
# loading in modules
import sqlite3
import sys
# from peewee import *
import pandas as pd 
from sudulunu.helpers import pp, make_num, dumper, rc

from transformers import pipeline

## Zero shot classifer
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")
# https://huggingface.co/facebook/bart-large-mnli

## Text classification/clustering
# https://huggingface.co/docs/transformers/tasks/sequence_classification

import os 
import pathlib
pathos = pathlib.Path(os.path.abspath('zero_shot.ipynb')).parent
os.chdir(pathos)
print(os.getcwd())


In [ ]:

db = 'input/scraperwiki.sqlite'
con = sqlite3.connect(db)

cur = con.cursor()

# table_list = [a for a in cur.execute("SELECT name FROM sqlite_master WHERE type = 'table'")]
# print(table_list)
# [('ads_by_id',), ('ads_by_query',)]

entries = [x for x in cur.execute('SELECT * FROM ads_by_query')]
# entries = [x for x in cur.execute('SELECT * FROM ads_by_id')]

# advertisers = []
# for thingo in entries:
#     advertisers.append(thingo[2])
# print(list(set(advertisers)))

# ['From the Heart', 'Fair Australia', 'The Uluru Statement from the Heart', 'Senator Jacinta Nampijinpa Price', 
# 'Not My Voice', 'Referendum News', 'Linda Burney', 'Not Enough', 'Yes23']

con.close()

In [1]:
records = []

candidate_labels = ['In favour of the Voice to Parliament', 'Against the Voice to Parliament']


counter = 0
for thingo in entries:
    counter += 1
    if counter % 50 ==0:
        
        print(counter)
    old = pd.read_csv('inter/zero_shot.csv')
    donners = old['Ad_id'].unique().tolist()
    ad_id = thingo[0]

    if ad_id in donners:
        continue
    else:

        try:

            advertiser_id = thingo[1]
            name = thingo[2]
            texto = thingo[4]
            classy = classifier(texto, candidate_labels)
            
            # print(ad_id)
            # print(classy['labels'][0])
            # print(texto)
            # print(classy['scores'])
            records.append({"Ad_id": ad_id, "Advertiser": advertiser_id, "Classification": classy['labels'][0], "Text": texto})

            cat = pd.DataFrame.from_records(records)
            new = pd.concat([old, cat])
            new.drop_duplicates(subset=['Ad_id'], inplace=True)

            dumper('inter', 'zero_shot', new)

        except Exception as e:

            print(f"Exception is {e}")
            print(f"Line: {sys.exc_info()[-1].tb_lineno}")
            pp(cat)

pp(cat)


NameError: name 'entries' is not defined

In [ ]:
zero = pd.read_csv('inter/zero_shot_yes_no.csv')

zero['Count'] = 1

grp = zero.groupby(by=['Classification'])['Count'].sum().reset_index()

print(grp)